### Tool Binding

[OpenAI Function calling](https://platform.openai.com/docs/guides/function-calling?api-mode=responses&example=get-weather) 참고
- OpenAI에서 제공하는 LLM은 함수를 호출해야 할 때를 감지한 후 함수를 호출하기 위한 인수가 포함된 JSON을 출력하도록 파인튜닝되어 있음

In [4]:
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

llm.invoke("오늘 서울 날씨가 어때?")

AIMessage(content='죄송하지만, 실시간 날씨 정보를 제공할 수는 없습니다. 서울의 현재 날씨를 확인하시려면 기상청 웹사이트나 날씨 앱을 이용해 보시기 바랍니다. 도움이 필요하시면 다른 질문 해주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 15, 'total_tokens': 67, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'stop', 'logprobs': None}, id='run-5b8b9406-8468-4e6f-97e8-0c042e6af1cc-0', usage_metadata={'input_tokens': 15, 'output_tokens': 52, 'total_tokens': 67, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

> LLM이 도구를 실행해주지는 않는다.

- 도구를 바인딩하고 모델을 호출하면, 답변을 위해 적절한 함수명과 params를 반환 

In [9]:
@tool
def get_weather(location: str) -> str:
    """Get the current weather in a given location"""
    return "오늘 서울 날씨는 맑아요"


tools = [get_weather]

llm_with_tools = llm.bind_tools(tools)

response = llm_with_tools.invoke("오늘 서울 날씨가 어때?")

print(response.tool_calls)

[{'name': 'get_weather', 'args': {'location': '서울'}, 'id': 'call_xaGu1AxfbTwMdA0yAEPRN2dF', 'type': 'tool_call'}]


In [21]:
toolbox = {"get_weather": get_weather}

tool_call = response.tool_calls[0]
tool_name = tool_call["name"]
tool_args = tool_call["args"]

result = toolbox[tool_name].invoke(tool_args)
print(result)

오늘 서울 날씨는 맑아요
